# Load and Subset NISAR GCOV data

## Option 1: Use `ISCE3`'s NISAR Product Reader

### This loads an HDF5 dataset as a `GenericProduct.GCOV` object.

In [ ]:
from nisar.products.readers import open_product
from pathlib import Path

gcov_path = Path('Real_NISAR_GCOV_VV_only/NISAR_L2_PR_GCOV_008_029_A_010_0005_NASV_A_20251031T044409_20251031T044425_P05000_N_P_J_001.h5')
gcov = open_product(gcov_path) 
gcov

 ### The GCOV object holds member variables containing product metadata.

In [ ]:
vars = [name for name in dir(gcov)
        if not callable(getattr(gcov, name)) and not name.startswith("_") and not name.startswith("pyre")]
max_len = max(len(var) for var in vars)

for var in vars:
    value = getattr(gcov, var)
    key = f"{var}:"
    print(f"gcov.{key:{max_len}} {value}")

### It also contains an `identification` object with additional metadata

In [ ]:
vars = [name for name in dir(gcov.identification)
        if not callable(getattr(gcov.identification, name)) and not name.startswith("_") and not name.startswith("pyre")]
max_len = max(len(var) for var in vars)

for var in vars:
    value = getattr(gcov.identification, var)
    key = f"{var}:"
    print(f"gcov.identification{key:{max_len}} {value}")

### `covarianceTerms` show which covariance bands and L-band frequencies are available

In [ ]:
gcov.covarianceTerms

:::{note} All possible covariance channels in quad-pol mode
:::{math}
C =
\begin{bmatrix}
\color{green}\langle HH\,HH^* \rangle 
  & \langle HH\,HV^* \rangle 
  & \langle HH\,VH^* \rangle 
  & \langle HH\,VV^* \rangle 
\\[6pt]
{\color{lightgray}\langle HV\,HH^* \rangle}
  & \color{green}\langle HV\,HV^* \rangle
  & \langle HV\,VH^* \rangle
  & \langle HV\,VV^* \rangle
\\[6pt]
{\color{lightgray}\langle VH\,HH^* \rangle}
  & {\color{lightgray}\langle VH\,HV^* \rangle}
  & \color{green}\langle VH\,VH^* \rangle
  & \langle VH\,VV^* \rangle
\\[6pt]
{\color{lightgray}\langle VV\,HH^* \rangle}
  & {\color{lightgray}\langle VV\,HV^* \rangle}
  & {\color{lightgray}\langle VV\,VH^* \rangle}
  & \color{green}\langle VV\,VV^* \rangle
\end{bmatrix}
\\[12pt]
\\[4pt]
\textcolor{black}{\text{Black}}:\ \text{Included off-diagonal covariance terms}\\
\textcolor{green}{\text{Green}}:\ \text{Included diagonal terms (backscatter)}\\
\textcolor{lightgray}{\text{Light gray}}:\ \text{Conjugate off-diagonal terms (not included)}
:::

### Load a backscatter image dataset

In [ ]:
vv_backscatter_power = gcov.getImageDataset(frequency='B', polarization='VVVV')
vv_backscatter_power

### Subset the data by index

In [ ]:
vv_subset = vv_backscatter_power[2000:2500, 2000:2500]
print(f"vv_subset.shape: {vv_subset.shape}")
vv_subset

### Perform statistics or transformations on the data

As a loaded HDF5 dataset, you can call many `numpy` functions directly on the data.

In [ ]:
import numpy as np

print(f"min: {np.nanmin(vv_backscatter_power)}")
print(f"max: {np.nanmax(vv_backscatter_power)}")
print(f"mean: {np.nanmean(vv_backscatter_power)}")

Note that some `numpy` functions and attributes are not available with the loaded HDF5 dataset. 

For example, the code cell below will raise an `AttributeError` when trying to view the `T` (transpose attribute)

In [ ]:
vv_backscatter_power.T

To view the `T` attribute, you must first read the data into memory as a `numpy.ndarray`. You can trigger the data to be read into memory by indexing the entire dataset with `[...]`:

`vv_backscatter_power.T` -> `vv_backscatter_power[...].T`



In [ ]:
print(f"vv_backscatter_power.shape: {vv_backscatter_power.shape}")
print(f"vv_backscatter_power[...].T.shape: {vv_backscatter_power[...].T.shape}\n")

vv_backscatter_power[...].T

## Option 2: Load the data with `xarray` using [utility functions](../util/load_gcov.py) included in this Cookbook

Loading the data as an `xarray.Dataset` provides access to `xarray`’s full toolset and makes it easy to work with lazily loaded, Dask-backed arrays.

### Find the paths to the GCOV data for the time series

In [ ]:
from pathlib import Path

gcov_paths = list(Path("/home/jovyan/NISAR_GCOV_Cookbook/notebooks/time_series_example/data").glob("*.h5"))
gcov_paths

### Load GCOV data into an xarray.Dataset with `load_gcov_ts_xr`

If you pass a single GCOV path to `load_gcov_ts_xr`, instead of a list of paths, it will create a time series Dataset with a single time step.

`load_gcov_ts_xr` lazily loads all raster data into xarray data structures with delayed HDF5 reads, so the data are not stored in memory until computed.

#### Load a single GCOV dataset

In [ ]:
import sys
from pathlib import Path

util_dir = Path.cwd().parent / "util"
sys.path.insert(0, str(util_dir))

from load_gcov import load_gcov_ts_xr

ds = load_gcov_ts_xr(gcov_paths[0])
ds

#### Access the backscatter raster


In [ ]:
vvvv = ds["VVVV"].isel(time=0, frequency=0)
vvvv

#### With the data in `xarray`, we can view its {abbr}`T (transpose)` attribute without having to first load it into memory 

In [ ]:
# vvvv transpose
vvvv.T

#### Load a GCOV time series

Pass a list of gcov paths instead of a single path.

In [ ]:
ds = load_gcov_ts_xr(gcov_paths)
ds

#### Access a single time step of the VVVV data

In [ ]:
vvvv = ds["VVVV"].sel(time="2025-10-31T04:44:09", frequency="B")
vvvv

#### Call the `xarray.mean` function

Notice that it does not provide a value. This is because the data is lazily loaded and has not yet been computed or stored to memory.

In [ ]:
vvvv_mean = vvvv.mean()
vvvv_mean

#### We can force the value to be computed by casting it to an appropriate datatype or calling `xarray.compute`

Note that computing does not cache the data.

In [ ]:
# cast to float
float(vvvv_mean)

In [ ]:
# call compute
vvvv_mean.compute()

### Subset the data at load-time

#### Load the time series of only the VVVV data for a single frequency in a subset spatial {abbr}`AOI (Area of Interest)`

In [ ]:
ds = load_gcov_ts_xr(
    gcov_paths, 
    vars_to_load=["VVVV"],
    freqs=["B"],
    y_slice=slice(1900000, 1800000),
    x_slice=slice(600000, 700000))
ds

### Plot the `VVVV` data with `xarray.plot`

#### Notice that due to outliers, most of the data are squeezed into a narrow portion of the colormap.

In [ ]:
vvvv = ds["VVVV"].isel(time=0, frequency=0)
vvvv.plot()

#### Set `vmin` and `vmax` to better scale the data across the colormap.

In [ ]:
vvvv = ds["VVVV"].isel(time=0, frequency=0)
vvvv.plot(vmin=0, vmax=0.0000000019)